In [1]:
#files to be imported
import numpy as np
from matplotlib import pyplot as plt

import os ,sys, platform
import time
import cosmopower as cp

In [2]:
#Planks Values
el = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 0, skiprows=1,dtype =float)
el_te = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 0, skiprows=1,dtype =float)
Dl_TT = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_TT_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_TT_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
Dl_EE = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_EE_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_EE_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
Dl_TE = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_TE_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_TE_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
#ells
dl_fac = el*(el+1)/(2*np.pi)
dl_fac_te = el_te*(el_te+1)/(2*np.pi)
#COSMOPOWER
tt_emu = cp.cosmopower_NN(restore=True,restore_filename='COSMO/cmb_TT_NN')
ee_emu = cp.cosmopower_NN(restore=True,restore_filename='COSMO/cmb_EE_NN')
te_emu = cp.cosmopower_PCAplusNN(restore=True,restore_filename='COSMO/cmb_TE_PCAplusNN')

In [3]:
def chi2(tre,teu,trb,pred): #chi^2 model
    #c = ((tre - pred)**2)/((pred)**2)
    c = ((tre - pred)**2)/((teu + trb)**2) # (Real - Model)^2/(Error Up + Error Bottom)^2
    chi = np.sum(c)
    return chi

#time ms to order of time
def time_hr(ms):
    ssm = ms*100
    seconds = int(ssm/100)
    minutes = int(seconds/60)
    hours = int(minutes/60)
    days = int(hours/24)
    sm = int(ssm - seconds*100)
    s = seconds - minutes*60 
    m = minutes - hours*60
    h = hours - days*24
    t = [sm,s,m,h,days]
    xi = ''
    if days == h == m == s == 0 :
        xi = str(sm) +'ms '
    elif days == h == m == 0:
        xi = str(s) +'s ' + str(sm) +' ms'
    elif days == h == 0:
        xi = str(m) + ' min ' + str(s) +'s ' + str(sm) +' ms'
    elif days == 0:
        xi = str(h) + 'hrs '+str(m) + 'min '+str(s) +'s ' + str(sm) +' ms'
    else :
        xi = str(days) +'days ' + str(h) + 'hrs '+str(m) + 'min '+str(s) +'s ' + str(sm) +' ms' 
    return xi #time 

def Find_val(inputrange,array):
    #finding the input value for the  minimum chi2 value
    min_chi2 = np.argmin(array) #position of minimum chi2
    vals = inputrange[min_chi2] #finding Input value of minimum chi2
    return vals

def Cosmo(xe,ze): # function for Cosmopower
    if xe == 'TT':
        tt_spectra = tt_emu.ten_to_predictions_np(ze)
        CO = tt_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
    elif xe == 'EE':
        ee_spectra = ee_emu.ten_to_predictions_np(ze)
        CA = ee_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
        CO = CA[:1995] #since planks data as only 1995 values 
    elif xe == 'TE':
        te_spectra = te_emu.predictions_np(ze)
        CA = te_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
        CO = CA[:1995] #since planks data as only 1995 values 
    else:
        Print('Check Input Range or Data')
    return CO

def Cosmo_Power(omb,omc,h,tau,ns,As): #function for single value cosmopower
    DET = 'TT'
    params = {'omega_b': [omb],'omega_cdm': [omc],'h': [h/100],'tau_reio': [tau],'n_s': [ns],'ln10^{10}A_s': [np.log( 10**10 *(As*1e-9) )],}
    Para = [omb,omc,h,tau,ns,As]
    if DET=='TT':
        pred = Cosmo('TT',params)*dl_fac
        chi = chi2(Dl_TT*dl_fac,Dl_TT_err_up*dl_fac,Dl_TT_err_bot*dl_fac,pred*dl_fac)
    elif DET=='TE':
        pred = Cosmo('TE',params)*dl_fac_te
        chi = chi2(Dl_TE,Dl_TE_err_up,Dl_TE_err_bot,pred)
    elif DET=='EE':
        pred = Cosmo('EE',params)*dl_fac_te
        chi = chi2(Dl_EE,Dl_EE_err_up,Dl_EE_err_bot,pred)
    reference = [omb,omc,h,tau,ns,As,chi]
    return chi #parameter, prediction , chi value

def Variable(max,min,var):#finds the variation
    return ((max-min)*var)
    
def array_generator(Par_vars): #generates an array for array of input
    para_array = []
    for i in range(2):#np.arange(Par_vars[0][1],Par_vars[0][0],Par_vars[0][2]):
        para_arrays = []
        for j in np.arange(Par_vars[1][1],Par_vars[1][0],Par_vars[1][2]):
            for k in np.arange(Par_vars[2][1],Par_vars[2][0],Par_vars[2][2]):
                for l in np.arange(Par_vars[3][1],Par_vars[3][0],Par_vars[3][2]):
                    for m in np.arange(Par_vars[4][1],Par_vars[4][0],Par_vars[4][2]):
                        for n in np.arange(Par_vars[5][1],Par_vars[5][0],Par_vars[5][2]):
                            ss = [i,j,k,l,m,n]
                            para_array.append(ss)
                            para_arrays.append(ss)
        np.save('Brute/Par_arr'+i,para_arrays)
    return para_array



In [4]:
#varing 6 parameters
omb_min = 0.01
omb_max = 0.04
omc_min = 0.01
omc_max = 0.4
h_min = 50
h_max = 90
tau_min = 0.01
tau_max = 0.09
ns_min = 0.3
ns_max = 1.5
As_min = 1.5
As_max = 3
Varying = 0.01

In [5]:
def random_par(parnumber,array,shapes=False):
    nr = np.arange(array[parnumber][1],array[parnumber][0],array[parnumber][2])
    shape = np.shape(nr)[0]
    random = np.random.randint(shape)
    par = nr[random]
    if shapes == True:
        return shape
    else:
        return par
def pos(parnumber,value,array):
    nr = np.arange(array[parnumber][1],array[parnumber][0],array[parnumber][2])
    index = np.where(nr==value)
    return index

def pos_par(par,array):
    poss = [] 
    for i in range(6):
        poss.append(pos(i,par[i],array)[0][0])
    position = poss[0]*poss[1]*poss[2]*poss[3]*poss[4]*poss[5]
    return position
    
    
def random_pars(count,par_array):
    array = []
    
    for j in range(count):
        par = []
        for i in range(6):
            par.append(random_par(i,par_array))
        array.append(par)
    return array
    
def pos_chi(count,par_arr):
    pos_chi = []
    for i in random_pars(10,Par_var):
        chi = Cosmo_Power(i[0],i[1],i[2],i[3],i[4],i[5])
        pso = pos_par(i,Par_var)
        pos_chi.append([pso,chi])
    return pos_chi

def likkili(pos_ch,Pa):
    sha = random_par(0,Pa,True)*random_par(1,Pa,True)*random_par(2,Pa,True)*random_par(3,Pa,True)*random_par(4,Pa,True)*random_par(5,Pa,True)
    shas = np.empty(sha)
    shas.fill(0)
    for i in pos_ch:
        shas[i[0]] = i[1]

    return shas

In [ ]:
Par_var = [[omb_max,omb_min,Variable(omb_max,omb_min,Varying)],
                 [omc_max,omc_min,Variable(omc_max,omc_min,Varying)],
                 [h_max,h_min,Variable(h_max,h_min,Varying)],
                 [tau_max,tau_min,Variable(tau_max,tau_min,Varying)],
                 [ns_max,ns_min,Variable(ns_max,ns_min,Varying)],
                 [As_max,As_min,Variable(As_max,As_min,Varying)]]


sr = array_generator(Par_var)

In [376]:
print(likkili(pss,Par_var))

MemoryError: Unable to allocate 6.94 EiB for an array with shape (1000000000000000000,) and data type float64

In [ ]:
np.save('Parameter_array_all_by_one_thousand',array_geb)

In [ ]:
def brute_force(array,lesa = False):
    reference = [[],[],[],[],[],[],[]]
    chi_array = []
    for i in array:
        chi = Cosmo_Power(i[0],i[1],i[2],i[3],i[4],i[5])
        for j in range(6):
            reference[j].append(i[j])
        reference[6].append(chi)
        chi_array.append(chi)
    least_par = [Find_val(reference[0],reference[6]),
                 Find_val(reference[1],reference[6]),
                 Find_val(reference[2],reference[6]),
                 Find_val(reference[3],reference[6]),
                 Find_val(reference[4],reference[6]),
                 Find_val(reference[5],reference[6]),]
    if lesa == True:
        return least_par,reference
    else : 
        return chi_array

<>:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\ELAKIA\AppData\Local\Temp\ipykernel_15892\2061005545.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  pars_aray = [['omb','omc']
C:\Users\ELAKIA\AppData\Local\Temp\ipykernel_15892\2061005545.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  pars_aray = [['omb','omc']
C:\Users\ELAKIA\AppData\Local\Temp\ipykernel_15892\2061005545.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  pars_aray = [['omb','omc']


TypeError: list indices must be integers or slices, not tuple